## TODO :
- fix the problem of else 
  - proposed solution `$` means any char , `#` any number  and `@` any thing does not exist in this dict
    - eg :
    - ```py
        0 :{ #...
          "<": 80,
        }
        80 :{
          ">":81,
          "=":82,
          "@": FINAL_STATE #no one of the previous
        }

      ```
  - second solution is to make `SAFE_FLAG` , if i got a nod that have 2 possibilities i use it intill pass the node
  - make in node validation (instead of put the final state in the previous node i will put it in the final node its self and ill put with it the other possibilities)

  

In [1]:
final_states = [
    # KEYWORDS
    "programme_FOUND", "constante_FOUND", "variable_FOUND", "debut_FOUND", "fin_FOUND", "répéter_FOUND", "jusqua_FOUND", # keywords
    
    # OPERATORS
    "RelationalOperator_L_FOUND", "RelationalOperator_G_FOUND", "RelationalOperator_LE_FOUND", 
    "RelationalOperator_GE_FOUND",'RelationalOperator_E_FOUND',"RelationalOperator_NE_FOUND",
    
    "AdditionOperator_PLUS_FOUND", "AdditionOperator_MINUS_FOUND", "AdditionOperator_OR_FOUND",
    
    "MultiplicationOperator_MULT_FOUND", "MultiplicationOperator_DIV_FOUND", "MultiplicationOperator_MOD_FOUND", "MultiplicationOperator_AND_FOUND",
    
    "Name_FOUND", "Integer_FOUND", "Real_FOUND", "Char_FOUND"
]

def lex(code):
    tokens = []
    state = 0
    buffer = ""  # Use a buffer instead of lexeme_start
    position = 0

    while position < len(code):
        char = code[position]
        next_state = rules.get(state, {}).get(char)

        if char.isspace():
            if state in final_states:
                tokens.append((final_states[state], buffer, position - len(buffer))) # use the buffer length to calculate the start position
            state = 0
            buffer = ""
        elif next_state is not None:
            state = next_state
            buffer += char # Add the current character to the buffer
        else:
            if state in final_states:
                tokens.append((final_states[state], buffer, position - len(buffer))) # use the buffer length to calculate the start position
            elif state != 0:
                print(f"Lexical Error: Unexpected character '{char}' after '{buffer}' at position {position}")
            state = 0
            buffer = ""
        position += 1

    if state in final_states:
        tokens.append((final_states[state], buffer, position - len(buffer))) # use the buffer length to calculate the start position
    return tokens

# Example rules and final states (you'll need to define these based on your language)
rules = {
    0: {'p': 1, 'v': 6, '+': 11, '1': 12, '4': 12},
    1: {'r': 2},
    2: {'o': 3},
    3: {'g': 4},
    4: {'r': 5},
    5: {'a': 10},
    6: {'a': 7},
    7: {'r': 8},
    8: {'i': 9},
    9: {'a': 10},
    10:{},
    11:{},
    12:{'1':12,'2':12,'3':12,'4':12,'5':12,'6':12,'7':12,'8':12,'9':12,'0':12,'.':13},
    13:{'1':14,'2':14,'3':14,'4':14,'5':14,'6':14,'7':14,'8':14,'9':14,'0':14},
    14:{'1':14,'2':14,'3':14,'4':14,'5':14,'6':14,'7':14,'8':14,'9':14,'0':14}
}
final_states = {10: "ID", 11: "PLUS", 12:"NUMBER", 14:"NUMBER"}

code = "programme variable x 123.5 + 456"
tokens = lex(code)
print(tokens)

code_with_error = "programme variable x 123$5"
tokens_with_error = lex(code_with_error)
print(tokens_with_error)

code_with_error2 = "programme variable x 123.5.5"
tokens_with_error2 = lex(code_with_error2)
print(tokens_with_error2)

[('ID', 'progra', 0), ('ID', 'varia', 10), ('NUMBER', '123.5', 21), ('PLUS', '+', 27), ('NUMBER', '456', 29)]
[('ID', 'progra', 0), ('ID', 'varia', 10), ('NUMBER', '123', 21)]
[('ID', 'progra', 0), ('ID', 'varia', 10), ('NUMBER', '123.5', 21)]


In [99]:
import string

final_states = {
    9: "Keyword_programme",
    18: "Keyword_constante",
    26: "Keyword_caractere",
    34: "Keyword_variable",
    39: "Keyword_debut",
    42: "Keyword_fin",
    49: "Keyword_répéter",
    51: "Keyword_réel",
    58: "Keyword_jusqua",
    64: "Keyword_entier",
    
    65: "MultOperator_et",
    68: "MultOperator_mod",
    71: "MultOperator_div",
    69: "MultOperator_mult",
    
    73: "AddOperator_ou",
    78: "AddOperator_plus",
    79: "AddOperator_minus",
    80: "RelOperator_L",
    81: "RelOperator_LE",
    82: "RelOperator_NE",
    83: "RelOperator_G",
    84: "RelOperator_GE",
    85: "RelOperator_E",
    
    86: "Semicolon",
    
    75: "Name",
    
    77: "Number",
    
    88: "Float",
    
    
    
}


# the states should be referred only by numbers 
rules = {
    0: {
        # Keywords
        "p": 1, "c": 10, "v": 27, "d": 35, "f": 40, "r": 43, "j": 52, ";": 86,
        
        # Relational Operators
        "<": 80, ">": 83, "=": 85,
        
        # Addition Operators
        "+": 78, 
        "-": 79,  
        "o": 72,
        
        # Multiplication Operators
        "e": 59, "m": 66, "*": 69, 
        
        # Name
        **{x: 74 for x in string.ascii_letters if x not in "pcvdfrjuoame"},
        
        # Numbers
        **{x: 77 for x in string.digits},
    },
    
    # Keywords
    1: {"r": 2},
    2: {"o": 3},
    3: {"g": 4},
    4: {"r": 5},
    5: {"a": 6},
    6: {"m": 7},
    7: {"m": 8},
    8: {"e": 9},  # programme_FOUND
    
    10: {"o": 11, "a": 19},
    11: {"n": 12},
    12: {"s": 13},
    13: {"t": 14},
    14: {"a": 15},
    15: {"n": 16},
    16: {"t": 17},
    17: {"e": 18},  # constante_FOUND
    
    19: {"r": 20},
    20: {"a": 21},
    21: {"c": 22},
    22: {"t": 23},
    23: {"è": 24},
    24: {"r": 25},
    25: {"e": 26},  # caractère_FOUND
    
    27: {"a": 28},
    28: {"r": 29},
    29: {"i": 30},
    30: {"a": 31},
    31: {"b": 32},
    32: {"l": 33},
    33: {"e": 34},  # variable_FOUND
    
    35: {"e": 36, "i": 70},
    36: {"b": 37},
    37: {"u": 38},
    38: {"t": 39},  # debut_FOUND
    70: {"v": 71},
    
    40: {"i": 41},
    41: {"n": 42},  # fin_FOUND
    
    43: {"é": 44, "e": 44},
    44: {"p": 45, "é": 50, "e": 50},
    45: {"é": 46, "e": 46},
    46: {"t": 47},
    47: {"e": 48},
    48: {"r": 48},  # répéter_FOUND
    
    50: {"l": 51},  # reel_FOUND
    
    52: {"u": 53},
    53: {"s": 54},
    54: {"q": 55},
    55: {"u": 56},
    56: {"'": 57},
    57: {"a": 58, "à": 58},  # jusqua_FOUND
    
    59: {"n": 60, "t": 65},
    60: {"t": 61},
    61: {"i": 62},
    62: {"e": 63},
    63: {"r": 64},  # entier_FOUND
    
    66: {"o": 67},
    67: {"d": 68},  # mod_FOUND
    
    72: {"u": 73},  # ou_FOUND
    
    87 :{**{x: 88 for x in string.digits}},
    
    
    # Final states
    **{i: {} for i in final_states.keys()},
    
    # overide the final states with other states
    74: {**{x: 75 for x in string.digits}},
    75: {**{x: 76 for x in string.ascii_lowercase}},
    76: {**{x: 75 for x in string.digits}},
    77: {**{x: 77 for x in string.digits}, ".": 87},
    80: {">": 82, "=": 81},
    83: {"=": 84},
    88: {**{x: 88 for x in string.digits}},
}


In [101]:


# this code is good it will work with some updates
def lex(code):
    code = code.lower() 
    tokens = []
    state = 0
    buffer = ""
    position = 0

    while position < len(code):
        char = code[position]
        next_state = rules.get(state, {}).get(char)
        
        print(f"> Loop {position} =======================================================")
        print(f"char: {char}, state: {state}, next_state: {next_state}, buffer: `{buffer}`")

        if char.isspace():
            print(f"    > char is space")
            if state in final_states:
                tokens.append((final_states[state], buffer, position - len(buffer)))
                print(f"        > in final state : append token {final_states[state]} ; `{buffer}` ; {position - len(buffer)}")
            state = 0
            buffer = ""
        elif next_state is not None:
            state = next_state
            buffer += char
            
            print(f"    > char is not space and next state = {state} (not None)")
        else:
            print(f"    > char is not space and next state is None")
            if state in final_states:
                print(f"        > in final state : append token {final_states[state]} ; `{buffer}` ; {position - len(buffer)}")
                tokens.append((final_states[state], buffer, position - len(buffer)))
                state = 0
                buffer = ""
                if char != " ":
                    position -=1
            elif state !=0:
                raise Exception(f"Lexical Error: Unexpected character '{char}' after `{buffer}` at position {position}")
            else :
                raise Exception(f"Lexical Error: Unexpected character '{char}' at position {position}")
            
            
        position += 1
        print(f"    > buffer: `{buffer}`")

    if state in final_states:
        tokens.append((final_states[state], buffer, position - len(buffer)))
    return tokens

code = "<2.2 22.34 >="
tokens = lex(code)
print(tokens)

> Loop 0 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 1 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 2 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 3 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 4 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 5 =======================================================
char:  , state: 0, next_state: None, buffer: ``
    > char is space
    > buffer: ``
> Loop 6 =======================================================
char:  , state: 0, next_state: None

In [96]:

keywords = ["programme;", "<\nconstante12x3", "variable", "debut", "fin", "répéter", "jusqu'à", "div", "mod", "et", "ou", "entier", "caractère", "réel"]
text = " ".join(keywords)
print(text)
tokens = lex(text)
print(tokens)

programme; <
constante12x3 variable debut fin répéter jusqu'à div mod et ou entier caractère réel
> Loop 0 =======================================================
char: p, state: 0, next_state: 1, buffer: ``
    > char is not space and next state = 1 (not None)
    > buffer: `p`
> Loop 1 =======================================================
char: r, state: 1, next_state: 2, buffer: `p`
    > char is not space and next state = 2 (not None)
    > buffer: `pr`
> Loop 2 =======================================================
char: o, state: 2, next_state: 3, buffer: `pr`
    > char is not space and next state = 3 (not None)
    > buffer: `pro`
> Loop 3 =======================================================
char: g, state: 3, next_state: 4, buffer: `pro`
    > char is not space and next state = 4 (not None)
    > buffer: `prog`
> Loop 4 =======================================================
char: r, state: 4, next_state: 5, buffer: `prog`
    > char is not space and next state = 5 (not 